In [2]:
# import cv2
# import matplotlib.pyplot as plt
# from PIL import Image
# from ultralytics import YOLO
import numpy as np
from pyproj import Proj, transform
from random import randint, random, choice

In [3]:
# model = YOLO('../best.pt')

In [ ]:
HALF_EQUATOR = 20037508.34

#### Задаем параметры камеры

In [17]:
wgs84 = Proj(init='epsg:4326')

d:\MyProgramms\Programs\Python\pothole-coordinates\.venv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [18]:
resolution = [i for i in range(400, 4000, 200)]
coefs = [i for i in range(10, 10000, 200)]

class Camera():
    def __init__(self):
        self.updateParam()
    
    def updateParam(self, h, az, ang, res, wc):
        self.height = h
        self.azimut = az
        self.angleView = ang
        self.resolition = res
        self.worldCoords = wc

#### Высчитываем мировые координаты ямы 

In [21]:
def epsg3857To4326(coords3857: np.array):
    lon = (coords3857.x / HALF_EQUATOR) * 180
    lat = (coords3857.y / HALF_EQUATOR) * 180
    lat = np.arctan(np.exp(lat * np.pi / 180))
    lat = lat / (Math.PI / 360)
    lat -= 90
    return np.array([lat, lon])

In [ ]:
def vectorWithAzimut(vec: np.array, azimut: float):
    quartMatrix = np.array([[np.cos(azimut),-np.sin(azimut)],
                             [np.sin(azimut),np.cos(azimut)]]) # Формирование матрицы
    return quartMatrix @ vec

In [ ]:
def convertCoords(coordsItemImage: np.array, cam: Camera):
    vectorPos = coordsItemImage / (cam.resolition[1] / 2)
    vectorPos = vectorWithAzimut(vectorPos, cam.azimut)
    vectorPos = vectorPos * cam.height + cam.worldCoords
    return epsg3857To4326(vectorPos)
        